# **The ChipTracker class**

This notebook provides information about ```ChipTracker``` objects.

Run the following codeblocks to import the ```ChipTracker``` class into this notebook.

In [1]:
from os import chdir, getcwd

if not getcwd().endswith("fivecarddraw"):
    chdir("..")
    
print(f"Current Directory: {getcwd()}")

Current Directory: c:\Users\Brad\Documents\Projects\Python\fivecarddraw


In [2]:
from fivecarddraw import ChipTracker

## **Initialisation**

The ```ChipTracker``` class is designed to be mediated by ```Dealer``` objects, so is created by ```Dealer``` objects upon initialisation. More info can be found in the [dealer.ipynb](dealer.ipynb) notebook. Upon initialisation of the ```ChipTracker``` object:
* It creates an attribute to track game parameters, such as the ante amount. This attribute is ```ChipTracker.gameinfo```.
* It creates attribute to track the chipstacks and any contributions to the pot made by players. This attribute is ```ChipTracker.players```

Both attributes store data as a ```dict``` to be fed elsewhere if required.

In [3]:
tracker = ChipTracker()
print(f"Game info: {tracker.gameinfo}")
print(f"Player info: {tracker.players}")

Game info: {'ante': 0}
Player info: {}


## **Functionality**

### **Assigning chips to players**

The recommended way to assign chips to a player is using ```ChipTracker.AddChipsPlayer(name, amount)``` which correctly updates ```ChipTracker.players```.

In [3]:
player = "Brad"

tracker = ChipTracker()
tracker.AddChipsPlayer(player, 500)
print(f"Player Chipstacks: {tracker.ChipStacks()}")

Player Chipstacks: {'Brad': 500}


```ChipTracker.WithdrawChipsPlayer(name)``` can be used to delete all information about a player.

In [4]:
tracker.WithdrawChipsPlayer(player)
print(f"Player Chipstacks: {tracker.ChipStacks()}")

Player Chipstacks: {}


### **Contributing chips to the pot**

The recommended way to contribute chips to a player is using ```ChipTracker.BetChipsPlayer(name, amount)``` which correctly updates ```ChipTracker.players``` to track both chipstacks and contributions.

In [7]:
player = "Brad"

tracker = ChipTracker()
tracker.AddChipsPlayer(player, 500)
print(f"Player Chips before bet: {tracker.players}")
tracker.BetChipsPlayer(player, 100)
print(f"Player Chips after bet: {tracker.players}")

Player Chips before bet: {'Brad': {'stack': 500, 'contribution': 0}}
Player Chips after bet: {'Brad': {'stack': 400, 'contribution': 100}}


To ensure a player has bet a legal amount, the ```ChipTracker.ApproveBet(name, amount)``` can be used. It asserts the ```amount``` is not larger than ```tracker.players[name]['stack']```.

In [8]:
player = "Brad"
bet = 700

tracker = ChipTracker()
tracker.AddChipsPlayer(player, 500)
print(f"Player Chips before bet: {tracker.players}")
print(f"Bet to approve: {bet}")
print(f"Approval: {tracker.ApproveBet(player, 700)}")

Player Chips before bet: {'Brad': {'stack': 500, 'contribution': 0}}
Bet to approve: 700
Approval: False


### **Rewarding chips from a pot**

```ChipTracker.RewardChipsPlayer(name, amount)``` can be used to add an amount of chips to a player.

**Warning:** ```ChipTracker.RewardChipsPlayer(name, amount)``` does not modify player contribution values, so it's recommended to clear contributions using ```ChipTracker.ClearPot()``` once all players have received their rewards.

In [12]:
player = "Brad"
opponent = "Phil"

tracker = ChipTracker()
tracker.AddChipsPlayer(player, 500)
tracker.AddChipsPlayer(opponent, 500)
tracker.BetChipsPlayer(player, 100)
tracker.BetChipsPlayer(opponent, 100)
print(f"Player Chips before rewarding: {tracker.players}")
tracker.RewardChipsPlayer(player, 200)
tracker.ClearPot()
print(f"Player Chips after rewarding, after pot is cleared: {tracker.players}")

Player Chips before rewarding: {'Brad': {'stack': 400, 'contribution': 100}, 'Phil': {'stack': 400, 'contribution': 100}}
Player Chips after rewarding, after pot is cleared: {'Brad': {'stack': 600, 'contribution': 0}, 'Phil': {'stack': 400, 'contribution': 0}}


The ```Dealer``` object is able to integrate data from the ```ChipTracker``` object with data from the ```HandTracker```, and ```ActionTracker``` objects into a ```dict``` containing ```player_info```, and use this to calculate rewards for players, taking into account:
* player pot contributions (```player_info[Player.name]["chips"]["contribution"]```)
* player action statuses (```player_info[Player.name]["status"]["has_folded"]```)
* player hand ranks (```player_info[Player.name]["hand"]["rank_n"]```)

Feeding this data into the ```ChipTracker.CalculateRewards(player_info)``` returns a ```dict``` of  ```rewards``` for each player. The ```ChipTracker.CalculateRewards(player_info)``` algorithm also handles split pots and side pot scenarios too. The algorithm for the ```ChipTracker.CalculateRewards(player_info)``` method works as follows:

1. A reward priority is assigned to each player by filtering out players who have folded, and sorting players by hand rank, from best to worst.
2. Players are grouped by hand ranks using ```itertools.groupby()``` which maintains order.
3. Players in each group are sorted by pot contribution, from smallest to largest.
4. For each player in the group, contributions are taken from other players, capped by the contribution of the player, and placed into a pot to split amongst the group, thus handling split pots. The player takes a share dependent on the amount of players in the group that have contributed the same or more than them, thus handling side pots too.

An interactive example of this algorithm is not presented in this notebook, but it's presented in the [dealer.ipynb](dealer.ipynb) notebook instead, because of the dependence of it on data from other objects created by the ```Dealer``` object. Furthermore, for more information about player statuses and hand ranks check out the [actiontracker.ipynb](actiontracker.ipynb) and [handtracker.ipynb](handtracker.ipynb) notebooks. For more information about the ```Dealer.Payout()``` and ```Dealer.PlayerInfo()``` methods checkout the [dealer.ipynb](dealer.ipynb).



### **Classifying transactions**

The ```ChipTracker.BetStatus(name, amount)``` method can be used to check a players proposed bet against a minimum required bet to call, and intepret it as a proposed action. It returns a ```dict``` with the following keys, which are assigned a ```bool```:

* ```has_raised``` indicates wether the player has proposed a raise.
* ```has_allin``` indicates wether the player has propsed to go allin.
* ```has_mincalled``` indicates wether the player has proposed to pay the minimum requrired to call.
* ```has_folded``` indicates wether the player has proposed to fold.

The minimum sized bet required to call can be calculated using ```ChipTracker.AmountToCall(name)```.

In [11]:
player = "Brad"
opponent = "Phil"
response = 300

tracker = ChipTracker()
tracker.AddChipsPlayer(opponent, 500)
tracker.AddChipsPlayer(player, 500)
tracker.BetChipsPlayer(opponent, 200)
print(f"{opponent} bet 200 chips.")
print(f"{player} needs to pay {tracker.AmountToCall(player)} chips to call.")
print(f"{player} wants to pay {response} chips.")
print(f"The bet status of {player}'s proposed bet is: {tracker.BetStatus(player, response)}")

Phil bet 200 chips.
Brad needs to pay 200 chips to call.
Brad wants to pay 300 chips.
The bet status of Brad's proposed bet is: {'has_raised': True, 'has_allin': False, 'has_mincalled': True, 'has_folded': False}


### **Handling the Ante**

The ante can be set using the ```Chiptracker.SetAnte()``` method, and it can be retrieved using the ```ChipTracker.GetAnte()``` method. The ante amount is stored in the ```Chiptracker.gameinfo["ante"]``` attribute. The ante can also be compared to a players chips using the ```Chiptracker.CheckAnte(player)``` to indicate whether they can pay the ante or not.

In [4]:
tracker = ChipTracker()

print(f"Ante before being set: {tracker.GetAnte()}")
tracker.SetAnte(50)
print(f"Ante after being set: {tracker.GetAnte()}")

Ante before being set: 0
Ante after being set: 50


In [5]:
name = "Brad"
amount = 500

tracker.AddChipsPlayer(name, amount)
print(f"Comparison of player to ante: {tracker.CheckAnte(name)}")

Comparison of player to ante: {'bet_all': False, 'bet_something': True, 'bet_nothing': False}


## **Additional Convenience Methods**

```ChipTracker``` objects have the following additional convenience methods:
* ```SeatTracker.PotTotal()``` - returns the sum of all contributions to the pot.
* ```SeatTracker.ChipStacks()``` - returns player chipstacks.

In [13]:
player = "Brad"
opponent = "Phil"

tracker = ChipTracker()
tracker.AddChipsPlayer(opponent, 500)
tracker.AddChipsPlayer(player, 500)
tracker.BetChipsPlayer(opponent, 200)
tracker.BetChipsPlayer(player, 200)
print(f"Player stacks: {tracker.ChipStacks()}")
print(f"Amount in the pot: {tracker.PotTotal()}")


Player stacks: {'Phil': 300, 'Brad': 300}
Amount in the pot: 400
